# UDACITY Designing Your First Workflow - Step Functions

## Step Functions & SageMaker

In the prior exercises, we've been working with many small services. This can be overwhelming for a data scientist that wants to establish a consistent methodology for handling data. Step Functions is an orchestration service that can allow us to utilize SageMaker in a methodical and consistent way. Step Functions also integrates with Lambda, which can allow us to potentially automate our entire machine learning pipeline end-to-end. Let's get a handle on what a 'step' in a step function looks like.

In this exercise, you will create a preprocessing step and a training step. Then you will create a step function to chain the two steps.

## Exercise: Grant Permissions and install packages.

Attach the IAMFullAccess and the StepFunctionsFullAccess polices to your SageMaker execution role.

In [23]:
%%bash
pip install stepfunctions

## Exercise: Fill out preprocessing step.

The 'step' interface is designed to be quite similar to the Preprocessing Job in lesson 2. The main difference between these is the ability of a 'step' to interface with other steps. Given the successful outcome of a single step, the next step specified in a workflow will automatically continue. In our case, a training step will launch given the successful outcome of a preprocessing step. The preprocessing step has been encoded for you. Upload the preprocessing code 'HelloBlazePreprocess.py' and the zipped dataset 'reviews_Musical_Instruments_5.json.zip' to s3, and fill out the constants in the code below. 

In [24]:
import boto3
import os
from botocore.exceptions import ClientError

BUCKET = "udacity-xin-mlworkflow"
s3_prefix = "lesson3/exercise3"

def upload_file_to_s3(file_name, s3_prefix):
    object_name = os.path.join(s3_prefix, file_name)
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, BUCKET, object_name)
    except ClientError as e:
        logging.error(e)
        return False

file_name = 'HelloBlazePreprocess.py'
upload_file_to_s3(file_name, s3_prefix)
print("file uploaded!")

file uploaded!


Code below is the preprocessing step. Fill in the constants in the code.

In [27]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from stepfunctions.steps.sagemaker import ProcessingStep
import sagemaker

role = get_execution_role()

print(role)
PREPROCESSING_JOB_NAME = "test-job6-preprocess"
input_data = 's3://udacity-xin-mlworkflow/lesson3/reviews_Musical_Instruments_5.json.zip'
input_preprocessing_code = 's3://udacity-xin-mlworkflow/lesson3/exercise3/HelloBlazePreprocess.py'
sess = sagemaker.Session()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)


processed_data_train = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_train_scikit')
processed_data_test = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_test_scikit')

inputs=[
    ProcessingInput(
        source=input_data, 
        destination='/opt/ml/processing/input', 
        input_name = 'input-1'),  
    ProcessingInput(
        source=input_preprocessing_code , 
        destination='/opt/ml/processing/input/code', 
        input_name = 'code')
]


outputs=[
    ProcessingOutput(
        source='/opt/ml/processing/output/train', 
        destination=processed_data_train, output_name = 'train_data'), 
    ProcessingOutput(
        source='/opt/ml/processing/output/test', 
        destination=processed_data_test, output_name = 'test_data')
]


processing_step = ProcessingStep(
    "SageMaker pre-processing step 4",
    processor=sklearn_processor,
    job_name=PREPROCESSING_JOB_NAME,
    inputs=inputs,
    outputs=outputs,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/HelloBlazePreprocess.py"],
)


arn:aws:iam::510096416228:role/service-role/AmazonSageMaker-ExecutionRole-20250321T133435


## Exercise: Fill out Training Step

Upon the success of the preprocessing step, we wish to execute a training step. A training step is defined below. Fill the constants in the code.

In [28]:
from stepfunctions.steps.sagemaker import TrainingStep
import boto3

WORKFLOW_OUTPUT = "s3://udacity-xin-mlworkflow/lesson3/exercise3/workflow_output"
TRAINING_JOB_NAME = "test-job2-train"

region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    region=region_name, framework="blazingtext", version="latest"
)

helloBlazeEstimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=WORKFLOW_OUTPUT,
    sagemaker_session=sess,
)

helloBlazeEstimator.set_hyperparameters(mode='supervised')

training_step = TrainingStep(
    "SageMaker Training Step",
    estimator=helloBlazeEstimator,
    data={"train": sagemaker.TrainingInput(processed_data_train, content_type="text/plain"), "validation": sagemaker.TrainingInput(processed_data_test, content_type="text/plain")},
    job_name=TRAINING_JOB_NAME,
    wait_for_completion=True,
)

## Exercise: Create Workflow & Execute It. 

To link the steps, you'll need to create a role that is capable of doing so. Go to IAM and create a Step Functions role, and attach the CloudWatchEventsFullAccess and SageMakerFullAccess policies. Once done, make use of the above steps to create a workflow. Quick debugging tip: jobs must have a unique name; you'll need to rename job names when debugging. Consider creating a method that will dynamically create unique job names! 

In [29]:
from stepfunctions.steps import Chain
from stepfunctions.workflow import Workflow

workflow_role = "arn:aws:iam::510096416228:role/xin-stepfunction"

workflow_graph = Chain([processing_step, training_step])
workflow = Workflow(
    name="xin-stepfunction-workflow",
    definition=workflow_graph,
    role=workflow_role,
)

workflow.create()

execution = workflow.execute(
    inputs={
        "PreprocessingJobName": PREPROCESSING_JOB_NAME,  # Each pre processing job (SageMaker processing job) requires a unique name,
        "TrainingJobName": TRAINING_JOB_NAME  # Each Sagemaker Training job requires a unique name,       
    }
)

execution_output = execution.get_output(wait=True)

You can track the outcome of this workflow through a custom UI that gets generated! Check it out!

In [30]:
execution.render_progress()